In [ ]:
#Importamos las liberías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import seaborn as sns
%matplotlib inline

from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
#import pydotplus
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

## PREPARACIÓN DE DATOS

In [ ]:
#Cargamos el csv
df=pd.read_csv("cancer_mama.csv",sep=",")
df

In [ ]:
#Ver los valores de cada columna
for i in df.columns:
    print(df[i].unique())

In [ ]:
#Vemos los valores que tiene cada columna y cuantas veces aparece cada uno
#de ellos
for i in df.columns:
    print(i,df[i].value_counts())

In [ ]:
#Información del dataset
df.info()

In [ ]:
#Guardamos el dataset como csv
df.to_csv("cancer_mama_py.csv", index=False)

In [ ]:
# Calcula la frecuencia de los valores de clase en todo el DataFrame
frecuencia_clase = df['diagnosis'].value_counts()

# Crea un gráfico de barras para visualizar la frecuencia de los valores 
#de clase
plt.bar(frecuencia_clase.index, frecuencia_clase.values, color="pink")
plt.xlabel('Diagnóstico')
plt.ylabel('Frecuencia')
plt.show()


# PREPROCESAMIENTO

In [ ]:
# Convertir diagnosis a numérico
df.diagnosis = df.diagnosis.replace({"B": 0, "M": 1})

# Dividimos entrada y salida
x=df.iloc[:,2:32]
print(x)
y=df[["diagnosis"]]
# Dividimos en datos de entrenamiento y de validación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

# Tamaños
x.shape, y.shape

x_train.shape, y_train.shape

x_test.shape, y_test.shape

# Calculamos el máximo y mínimo
maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)

ranges = maxs - mins

# Normalizamos las variables
x_train = (x_train - mins) / ranges
x_test = (x_test - mins) / ranges

# Calcula las frecuencias de clase en y_train
class_frequencies = y_train.value_counts()

# Calcula los pesos para cada clase  
class_weights = {cls: 1.0 / freq for cls, freq in class_frequencies.items()}
class_weights_int = {int(cls[0]): weight for cls, weight in class_weights.items()}

print(class_frequencies)
print(class_weights_int)

pesos_ordenados=[]
for i in range(2):
    peso=class_weights_int[i]
    pesos_ordenados.append(peso)

print(pesos_ordenados)
print(class_frequencies.values * pesos_ordenados)
# Crea un gráfico de barras para visualizar la frecuencia ponderada de los valores de clase
plt.bar([0,1], class_frequencies.values * pesos_ordenados, color= "pink")
plt.xlabel('Diagnóstico')
plt.ylabel('Ponderado')
plt.show()

# MODELOS

## Árbol de decisión

In [ ]:
# modelo genérico
DT = DecisionTreeClassifier(class_weight=class_weights_int)
# entrenamos el modelo
DT.fit(x_train, y_train)
# hacer predicciones
pred_DT = DT.predict(x_test)
# informe
print(classification_report(y_test,pred_DT))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_prec=score.mean()
print("cross_value: ", DT_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parametros
DT_combinaciones={}
# parametros
splitter_ = ["best", "random"]
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for s in splitter_:
            for p in [1, 2]:
                vector_parametros=[c,m,s,porc_[p-1]]
                # dividimos las instancias
                x_train, x_test, y_train, y_test = train_test_split(
                    x, y, test_size=porc_[p - 1], random_state=42, shuffle=True
                )
                # modelo
                DT = DecisionTreeClassifier(criterion=c, max_features=m, splitter=s, class_weight=class_weights_int)
                # entrenamos el modelo
                DT.fit(x_train, y_train)
                # hacer predicciones
                predicciones_DT = DT.predict(x_test)
                # calculamos la exactitud del modelo
                acc_1 = accuracy_score(y_test, predicciones_DT)
                # cross value
                kfold = StratifiedKFold(10)
                score = cross_val_score(DT, x_train, y_train, cv=kfold, scoring="accuracy")
                print(
                    c,
                    ",",
                    m,
                    "y",
                    s,
                    "(",
                    porc_[p - 1],
                    ")",
                    "-> accuracy_score: ",
                    acc_1,
                    "//",
                    "cross_value: ",
                    score.mean()
                )
                # añadimos al diccionario esta combinación
                DT_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
DT_max_prec = max(DT_combinaciones)
# miramos con que parámetros se consigue esta precisión
DT_mejor_vector_parametros = DT_combinaciones[DT_max_prec]
print("mejores resultados con los parametros -> ",DT_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
DT_mejor = DecisionTreeClassifier(criterion=DT_mejor_vector_parametros[0], max_features=DT_mejor_vector_parametros[1], splitter=DT_mejor_vector_parametros[2], class_weight=class_weights_int)
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=DT_mejor_vector_parametros[3], random_state=42, shuffle=True)
# entrenamos el modelo
DT_mejor.fit(x_train, y_train)
# hacer predicciones
pred_DT_mejor = DT_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_DT_mejor))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_mejor_prec=score.mean()
print("cross_value: ", DT_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Random forest

In [ ]:
# modelo genérico
y_ravel= np.ravel(y)
RF = RandomForestClassifier(class_weight=class_weights_int)
# entrenamos el modelo
y_train=y_train.values.ravel()
RF.fit(x_train, y_train)
# hacer predicciones
pred_RF = RF.predict(x_test)
# informe
print(classification_report(y_test,pred_RF,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_prec=score.mean()
print("cross_value: ", RF_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parámetros
RF_combinaciones={}
# parametros
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for p in [1, 2]:
            vector_parametros=[c,m,porc_[p-1]]
            # dividimos las instancias
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=porc_[p - 1], random_state=42, shuffle=True)
            y_train=y_train.values.ravel()
            # modelo
            RF = RandomForestClassifier(criterion=c, max_features=m, class_weight=class_weights_int)
            # entrenamos el modelo
            RF.fit(x_train, y_train)
            # hacer predicciones
            predicciones_RF = RF.predict(x_test)
            # calculamos la exactitud del modelo
            acc_1 = accuracy_score(y_test, predicciones_RF)
            # cross value
            kfold = StratifiedKFold(10)
            score = cross_val_score(RF, x_train, y_train, cv=kfold, scoring="accuracy")
            print(
                c,
                "y",
                m,
                "(",
                porc_[p - 1],
                ")",
                "-> accuracy_score: ",
                acc_1,
                "//",
                "cross_value: ",
                score.mean()
            )
            # añadimos al diccionario esta combinación
            RF_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
RF_max_prec = max(RF_combinaciones)
# miramos con que parámetros se consigue esta precisión
RF_mejor_vector_parametros = RF_combinaciones[RF_max_prec]
print("mejores resultados con los parametros -> ",RF_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
RF_mejor = RandomForestClassifier(criterion=RF_mejor_vector_parametros[0], max_features=RF_mejor_vector_parametros[1], class_weight=class_weights_int)
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=RF_mejor_vector_parametros[2], random_state=42, shuffle=True)
y_train=y_train.values.ravel()
# entrenamos el modelo
RF_mejor.fit(x_train, y_train)
# hacer predicciones
pred_RF_mejor = RF_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_RF_mejor))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_mejor_prec=score.mean()
print("cross_value: ", RF_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Redes neuronales

In [ ]:
optim = ["adam", "SGD", "RMSprop", "adagrad", "adamax"]
RN_combinaciones = {}

# Separa los atributos y la variable de salida
x = df.iloc[:, 2:32].values  
y = df.iloc[:, 1].values  

# Divide los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Escala los atributos para normalizarlos
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# ESTRUCTURA 1
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(30,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.round(y_pred_prob)
    y_pred = np.round(np.nan_to_num(y_pred))
    
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision]=[1,i]
    
# ESTRUCTURA 2
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(128, activation='relu', input_shape=(30,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.round(y_pred_prob)
    y_pred = np.round(np.nan_to_num(y_pred))
    
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision] = [2, i]
    
# ESTRUCTURA 3
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(30,)))  # Capa oculta 1
RNmodel.add(Dense(128, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 4
RNmodel.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.round(y_pred_prob)
    y_pred = np.round(np.nan_to_num(y_pred))
  
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision] = [3, i]
    
print(RN_combinaciones)
# Calculamos la precisión máxima
RN_max_prec = max(RN_combinaciones)
# Miramos con qué parámetros se consigue esta precisión
RN_mejor_vector_parametros = RN_combinaciones[RN_max_prec]
print("Mejores resultados con los parámetros:", RN_mejor_vector_parametros)


In [ ]:
# Mejor modelo

RNmodel_mejor = Sequential()

if RN_mejor_vector_parametros[0]==1:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(30,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)
elif RN_mejor_vector_parametros[0]==2:
    RNmodel_mejor.add(Dense(128, activation='relu', input_shape=(30,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)
elif RN_mejor_vector_parametros[0]==3:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(30,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(128, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 4
    RNmodel_mejor.add(Dense(1, activation='sigmoid'))  # Capa de salida (1 nivel de salida)
# Compila el modelo
RNmodel_mejor.compile(optimizer=RN_mejor_vector_parametros[1], loss='binary_crossentropy', metrics=['accuracy'])

# Entrena el modelo
RNmodel_mejor.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

y_pred_prob = RNmodel_mejor.predict(x_test)
y_pred = np.round(y_pred_prob)

# Calcular precisión
RN_mejor_prec = accuracy_score(y_test, y_pred)
print("Precisión:", RN_mejor_prec)

# Calcular matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
ConfusionMatrixDisplay(matriz_confusion).plot()
plt.show()

## KNN

In [ ]:
avg_score=[]
max_score=0
best_k=0
for k in range(3,30):
    knn_model=KNeighborsClassifier(n_jobs=-1,n_neighbors=k)
    score=cross_val_score(knn_model,x_train,y_train,n_jobs=-1,cv=5,scoring='accuracy')
    if score.mean()>max_score:
        max_score=score.mean()
        best_k=k
    avg_score.append(score.mean())

print([best_k,max(avg_score)])

max_index = avg_score.index(max(avg_score))
max_value = max(avg_score)

plt.figure(figsize=(15, 8))
plt.plot(range(3, 30), avg_score, color='magenta', linewidth=2)  
plt.scatter(max_index+2, max_value, color='purple', marker='o', s=200)  
plt.xlabel("k neighbors")
plt.ylabel("accuracy")

plt.show()

In [ ]:
# modelo
KNN=KNeighborsClassifier(n_neighbors=best_k)
# entrenamos el modelo
KNN.fit(x_train,y_train)
# hacer predicciones
pred_KNN = KNN.predict(x_test)
# informe
print(classification_report(y_test,pred_KNN,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    KNN, x_train, y_train, cv=kfold, scoring="accuracy"
)
KNN_prec=score.mean()
print("cross_value: ", KNN_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_KNN)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

# SVG

In [ ]:
# modelo
modelsvc = SVC()
# entrenamos el modelo
modelsvc.fit(x_train, y_train)
# hacer predicciones
pred_SVC = modelsvc.predict(x_test)
# informe
print(classification_report(y_test,pred_SVC,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    modelsvc, x_train, y_train, cv=kfold, scoring="accuracy"
)
SVC_prec=score.mean()
print("cross_value: ", SVC_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_SVC)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parámetros
svc_combinaciones={}
# parametros
c_ = [1, 20, 100, 200]
kernel_ = ["linear", "poly", "rbf", "sigmoid"]
porc_ = [0.2, 0.3]
# combinaciones
for c in c_:
    for k in kernel_:
        for p in [1, 2]:
            vector_parametros=[c,k,porc_[p-1]]
            # dividimos las instancias
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=porc_[p - 1], random_state=42, shuffle=True)
            # modelo
            modelsvc = SVC(C=c, kernel=k, class_weight=class_weights_int)
            # entrenamos el modelo
            modelsvc.fit(x_train, y_train)
            # hacer predicciones
            predicciones_SVC = modelsvc.predict(x_test)
            # calculamos la exactitud del modelo
            acc_1 = accuracy_score(y_test, predicciones_SVC)
            # cross value
            kfold = StratifiedKFold(10)
            score = cross_val_score(modelsvc, x_train, y_train, cv=kfold, scoring="accuracy")
            print(
                c,
                "y",
                k,
                "(",
                porc_[p - 1],
                ")",
                "-> accuracy_score: ",
                acc_1,
                "//",
                "cross_value: ",
                score.mean()
            )
            # añadimos al diccionario esta combinación
            svc_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
svc_max_prec = max(svc_combinaciones)
# miramos con que parámetros se consigue esta precisión
svc_mejor_vector_parametros = svc_combinaciones[svc_max_prec]
print("mejores resultados con los parametros -> ",svc_mejor_vector_parametros) 

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
svc_mejor = SVC(C=svc_mejor_vector_parametros[0], kernel=svc_mejor_vector_parametros[1], class_weight=class_weights_int)
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=svc_mejor_vector_parametros[2], random_state=42, shuffle=True)
# entrenamos el modelo
svc_mejor.fit(x_train, y_train)
# hacer predicciones
pred_svc_mejor = svc_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_svc_mejor,zero_division=1))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    svc_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
svc_mejor_prec=score.mean()
print("cross_value: ", svc_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_svc_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

# DIAGNOSIS

In [ ]:
nombre_v=str(input("Ahora introduciras los datos médicos del paciente con nombre o identificador: "))
radius_mean_v=float(input("radio (media): "))
texture_mean_v=float(input("textura (media): "))
perimeter_mean_v=float(input("perimetro (media): "))
area_mean_v=float(input("area (media): "))
smoothness_mean_v=float(input("suavidad (media): "))
compactness_mean_v=float(input("compacidad (media): "))
concavity_mean_v=float(input("concavidad (media): "))
concave_points_mean_v=float(input("puntos cóncavos (media): "))
symmetry_mean_v=float(input("simetría (media): "))
fractal_dimension_mean_v=float(input("dimensión fractal (media): "))
radius_se_v=float(input("radio (se): "))
texture_se_v=float(input("textura (se): "))
perimeter_se_v=float(input("perimetro (se): "))
area_se_v=float(input("area (se): "))
smoothness_se_v=float(input("suavidad (se): "))
compactness_se_v=float(input("compacidad (se): "))
concavity_se_v=float(input("concavidad (se): "))
concave_points_se_v=float(input("puntos cóncavos (se): "))
symmetry_se_v=float(input("simetría (se): "))
fractal_dimension_se_v=float(input("dimensión fractal (se): "))
radius_worst_v=float(input("radio (worst): "))
texture_worst_v=float(input("textura (worst): "))
perimeter_worst_v=float(input("perimetro (worst): "))
area_worst_v=float(input("area (worst): "))
smoothness_worst_v=float(input("suavidad (worst): "))
compactness_worst_v=float(input("compacidad (worst): "))
concavity_worst_v=float(input("concavidad (worst): "))
concave_points_worst_v=float(input("puntos cóncavos (worst): "))
symmetry_worst_v=float(input("simetría (worst): "))
fractal_dimension_worst_v=float(input("dimensión fractal (worst): "))

In [ ]:
pac=[radius_mean_v, texture_mean_v, perimeter_mean_v, area_mean_v, smoothness_mean_v, compactness_mean_v, concavity_mean_v, concave_points_mean_v, symmetry_mean_v, fractal_dimension_mean_v, radius_se_v, texture_se_v, perimeter_se_v, area_se_v, smoothness_se_v, compactness_se_v, concavity_se_v, concave_points_se_v, symmetry_se_v, fractal_dimension_se_v, radius_worst_v, texture_worst_v, perimeter_worst_v, area_worst_v, smoothness_worst_v, compactness_worst_v, concavity_worst_v, concave_points_worst_v, symmetry_worst_v, fractal_dimension_worst_v]
#pac=[17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189]
#nombre_v="magali"

# Crear los DataFrames que necesitaremos
df_diagnostico = pd.DataFrame(columns=['Método', 'Características', 'Precisión', 'Diagnóstico'])

df_datos=pd.DataFrame({'medida':['radio(media)', 'textura(media)', 'perimetro(media)', 'area(media)', 'suavidad(media)', 'compacidad(media)', 'concavidad(media)', 'puntos_concavos(media)', 'simetria(media)', 'dimesion_fractal(media)', 'radio(se)', 'textura(se)', 'perimetro(se)', 'area(se)', 'suavidad(se)', 'compacidad(se)', 'concavidad(se)', 'puntos_concavos(se)', 'simetria(se)', 'dimesion_fractal(se)','radio(worst)', 'textura(worst)', 'perimetro(worst)', 'area(worst)', 'suavidad(worst)', 'compacidad(worst)', 'concavidad(worst)', 'puntos_concavos(worst)', 'simetria(worst)', 'dimesion_fractal(worst)'],
                     'valor':pac})

df_nombre=pd.DataFrame({'nombre':[nombre_v]})

# Predecir diagnóstico con diferentes métodos
DT_predict=int(DT_mejor.predict([pac]))
RF_predict=int(RF_mejor.predict([pac]))
pac_scaled = scaler.transform([pac])  # Escala el vector pac
y_pred_prob = RNmodel_mejor.predict(pac_scaled)
RN_predict = np.argmax(y_pred_prob, axis=1)
KNN_predict=int(KNN.predict([pac]))
svc_predict=int(svc_mejor.predict([pac]))

df_diagnostico = df_diagnostico.append({'Método': 'Árbol de decisión', 'Características': DT_mejor_vector_parametros, 'Precisión': DT_mejor_prec, 'Diagnóstico': DT_predict}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Random Forest', 'Características': RF_mejor_vector_parametros, 'Precisión': RF_mejor_prec, 'Diagnóstico': RF_predict}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Redes neuronales', 'Características': RN_mejor_vector_parametros, 'Precisión': RN_mejor_prec, 'Diagnóstico': RN_predict[0]}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'KNN', 'Características': 'K='+ str(best_k), 'Precisión': KNN_prec, 'Diagnóstico': KNN_predict}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'SVC', 'Características': svc_mejor_vector_parametros, 'Precisión': svc_mejor_prec, 'Diagnóstico': svc_predict}, ignore_index=True)

df_diagnostico


In [ ]:
#Guardamos los dataset como csv
df_datos.to_csv("datospac_cancer_mama_py.csv", index=False)
df_diagnostico.to_csv("diagnostico_cancer_mama_py.csv", index=False)
df_nombre.to_csv("nombrepac_cancer_mama_py.csv", index=False)
